# Bert experimenty

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf
from torch.utils.data import Dataset
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
from typing import Dict, List
import transformers
from transformers import AutoModel, BertTokenizerFast,BertTokenizer, DistilBertTokenizerFast
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM

In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install transformers==4.30

## Načítanie a príprava dát

In [ ]:
dataset = pd.read_csv('../Data/final_dataset_5.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['processed_text'], dataset['author_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)
y_test = label_encoder.transform(y_test)

## Bert

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
x_train = list(X_train)
x_val = list(X_valid)
x_test = list(X_test)

y_train = list(y_train)
y_val = list(y_valid)
y_test = list(y_test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_valid)
test_dataset = CustomDataset(test_encodings, y_test)

In [ ]:
labels_count = len(dataset['author_id'].unique())
print(labels_count)

81


## Experiment 1 - learning_rate = 5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=5e-5,             #default
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,4.102560,0.134752,0.011044,0.019493,0.007213
2,No log,3.732187,0.283688,0.043961,0.064327,0.046355
3,No log,3.238327,0.368794,0.111086,0.153300,0.118714
4,No log,2.707570,0.425532,0.183569,0.219691,0.183458
5,No log,2.428386,0.475177,0.218826,0.280445,0.230507
6,No log,2.161793,0.510638,0.251850,0.294562,0.263654
7,3.286100,2.002134,0.588652,0.314390,0.382384,0.331716
8,3.286100,1.876482,0.574468,0.320671,0.362749,0.326815
9,3.286100,1.787515,0.560284,0.289737,0.342262,0.299432
10,3.286100,1.740719,0.553191,0.312253,0.361892,0.321010


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=960, training_loss=2.2555461247762043, metrics={'train_runtime': 675.1285, 'train_samples_per_second': 22.52, 'train_steps_per_second': 1.422, 'total_flos': 4003177963696128.0, 'train_loss': 2.2555461247762043, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[-0.96875   , -0.3334961 , -0.62597656, ..., -0.68359375,
        -0.22729492,  7.0195312 ],
       [-0.46289062, -1.0068359 , -0.17382812, ...,  0.07861328,
        -0.14099121, -0.5463867 ],
       [-0.5703125 , -0.62060547, -0.18884277, ...,  1.5859375 ,
        -0.12939453, -2.0214844 ],
       ...,
       [-0.14562988,  0.15344238,  0.20947266, ...,  0.25976562,
        -0.80566406,  1.7832031 ],
       [-0.2541504 , -0.28881836,  1.4296875 , ...,  0.08587646,
        -0.19665527, -1.6142578 ],
       [-0.984375  , -0.48339844, -0.6152344 , ..., -0.80566406,
        -0.17858887,  7.1914062 ]], dtype=float32), label_ids=array([27, 31, 24, 50, 80,  1, 35, 80, 58, 80, 55, 57, 80, 44, 52, 66, 80,
       52,  6, 21, 55, 36, 50, 37, 29, 23, 77, 35, 70, 74, 35, 30, 53,  4,
       77, 52,  9, 61, 22, 80, 71, 56, 19, 17, 52, 51, 72, 11, 72, 74, 79,
       72,  3,  8, 53, 80,  8,  6, 72, 36,  9, 19, 77, 36,  6, 16, 70,  8,
       24, 65, 76, 80, 35, 80, 7

## Experiment 4 - learning_rate = 8e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=8e-5,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,4.125541,0.148936,0.007214,0.023392,0.009531
2,No log,3.518312,0.283688,0.031971,0.061404,0.039771
3,No log,2.792434,0.404255,0.178386,0.215859,0.176896
4,No log,2.425313,0.468085,0.231326,0.272031,0.227980
5,No log,2.220287,0.517730,0.257806,0.336745,0.269761
6,No log,1.977270,0.574468,0.355603,0.443301,0.373230
7,2.940300,1.716498,0.595745,0.338875,0.437719,0.360922
8,2.940300,1.718668,0.588652,0.404664,0.468384,0.402641
9,2.940300,1.628499,0.652482,0.447760,0.507552,0.451558
10,2.940300,1.621610,0.609929,0.398746,0.453001,0.397092


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precis

TrainOutput(global_step=960, training_loss=1.799149513244629, metrics={'train_runtime': 712.0189, 'train_samples_per_second': 21.353, 'train_steps_per_second': 1.348, 'total_flos': 4003177963696128.0, 'train_loss': 1.799149513244629, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[-0.30859375,  0.25708008, -1.6103516 , ..., -0.86621094,
         0.42016602,  4.9570312 ],
       [ 0.05029297, -0.8222656 , -0.08312988, ..., -0.97802734,
        -2.8769531 , -1.9902344 ],
       [-0.7583008 , -0.16137695,  0.56103516, ...,  2.5605469 ,
        -0.07562256, -0.7182617 ],
       ...,
       [-0.5229492 ,  1.9208984 , -0.80615234, ..., -0.57470703,
         0.08502197,  1.8369141 ],
       [-0.4416504 ,  0.09790039,  1.9003906 , ..., -0.01345825,
        -2.6503906 , -2.6816406 ],
       [-0.19238281,  0.30859375, -0.68896484, ..., -1.9335938 ,
         0.5913086 ,  8.625     ]], dtype=float32), label_ids=array([27, 31, 24, 50, 80,  1, 35, 80, 58, 80, 55, 57, 80, 44, 52, 66, 80,
       52,  6, 21, 55, 36, 50, 37, 29, 23, 77, 35, 70, 74, 35, 30, 53,  4,
       77, 52,  9, 61, 22, 80, 71, 56, 19, 17, 52, 51, 72, 11, 72, 74, 79,
       72,  3,  8, 53, 80,  8,  6, 72, 36,  9, 19, 77, 36,  6, 16, 70,  8,
       24, 65, 76, 80, 35, 80, 7